In [5]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
file_path = '/content/drive/MyDrive/hw/causal/chicago-police/' # change this

# Load the datasets
complaints_allegation = pd.read_csv(file_path + 'complaints-allegation.csv')
officers = pd.read_csv(file_path + 'final-profiles.csv')
salary = pd.read_csv(file_path + 'salary_2002-2017_2017-09.csv')
salary_ranks = pd.read_csv(file_path + 'salary-ranks_2002-2017_2017-09.csv')
merged_data_1 = pd.merge(salary_ranks, complaints_allegation, left_on='link_UID', right_on='link_UID', how='left')
merged_data = pd.merge(merged_data_1, officers, left_on='link_UID', right_on='link_UID', how='left')
# selected_columns = merged_data[['link_UID', 'race','gender','birth_year','salary', 'appointed_date', 'cleaned_rank_x','allegation_category','final_finding','penalty_code','number_of_days']]
# selected_columns.sample(n=10)
# sustained_df = merged_data[merged_data['final_finding'] == 'SUSTAINED']##
outcome_counts = merged_data['allegation_category'].value_counts(dropna=False)
# outcome_counts.head(10)
category_mapping = {
    'MISCELLANEOUS': 'Non-Violent',
    'DOMESTIC ALTERCATION - PHYSICAL ABUSE': 'Violent',
    'NEGLECT OF DUTY': 'Non-Violent',
    'EXCESSIVE FORCE - USE OF FIREARM / OFF DUTY - INJURY': 'Violent',
    'UNNECESSARY DISPLAY OF WEAPON / ON DUTY': 'Non-Violent',
    'USE OF PROFANITY': 'Non-Violent',
    'EXCESSIVE FORCE / OFF DUTY - NO INJURY': 'Violent',
    'EXCESSIVE FORCE / On DUTY - INJURY': 'Violent',
    'CONDUCT UNBECOMING': 'Non-Violent',
    'NO INJURY': "Non-Violent",
    'PUSH/PULL/GRAB': "Violent",
    'INJURY': "Violent",
    'FALSE': "Non-Violent",
    'FAIL TO SUBMIT': "Non-Violent",
    'INADEQUATE / FAILURE TO PROVIDE SERVICE': "Non-Violent",
    'HARASSMENT': "Non-Violent",
    'EXCESSIVE FORCE - USE OF FIREARM / OFF DUTY - NO INJURY': "Violent",
    'VERBAL ABUSE': "Non-Violent",
    'FAIL TO OBTAIN A COMPLAINT REGISTER NUMBER': "Non-Violent",
    'LEAVING ASSIGNMENT (DISTRICT, BEAT, SECTOR, COURT)': "Non-Violent",
    'INTOXICATED OFF DUTY': "Non-Violent",
    'CLOSED HAND STRIKE (PUNCH)': "Violent",
}
# sustained_df['violence_classification'] = sustained_df['allegation_category'].map(category_mapping).fillna('Non-Violent')
merged_data.loc[:, 'violence_classification'] = merged_data['allegation_category'].map(category_mapping).fillna('Non-Violent')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-5-05b25de609bf>:8: DtypeWarning: Columns (16,18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_allegation = pd.read_csv(file_path + 'complaints-allegation.csv')
<ipython-input-5-05b25de609bf>:9: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  officers = pd.read_csv(file_path + 'final-profiles.csv')


In [6]:
selected_columns = merged_data[['link_UID', 'race','gender','birth_year','salary', 'appointed_date', 'violence_classification', 'cleaned_rank_x','allegation_category','final_finding','penalty_code','number_of_days']]
selected_columns.sample(n=10)

,link_UID,race,gender,birth_year,salary,appointed_date,violence_classification,cleaned_rank_x,allegation_category,final_finding,penalty_code,number_of_days
667141,15891.0,WHITE,MALE,1970.0,99978.0,1995-06-05,Violent,DETECTIVE,EXCESSIVE FORCE - USE OF FIREARM / OFF DUTY - ...,NOT SUSTAINED,NaN,NaN
215122,8898.0,WHITE,MALE,1974.0,50538.0,2001-10-29,Non-Violent,POLICE OFFICER,MISCELLANEOUS,NOT SUSTAINED,NaN,NaN
945825,30893.0,WHITE,MALE,1975.0,70656.0,1999-05-10,Non-Violent,POLICE OFFICER,USE OF PROFANITY,UNFOUNDED,NaN,NaN
905914,18591.0,WHITE,MALE,1978.0,82788.0,2002-10-28,Non-Violent,POLICE OFFICER,MISCELLANEOUS,NOT SUSTAINED,NaN,NaN
846159,25445.0,HISPANIC,MALE,1956.0,60600.0,1989-12-26,Non-Violent,POLICE OFFICER,EXCESSIVE FORCE / ON DUTY - INJURY,NOT SUSTAINED,NaN,NaN
1098631,15913.0,WHITE,MALE,1966.0,76446.0,1995-12-04,Non-Violent,DETECTIVE,NEGLECT OF DUTY,SUSTAINED,NaN,NaN
1199613,32366.0,WHITE,MALE,1971.0,86520.0,1998-06-29,Non-Violent,POLICE OFFICER,TWISTED EXTREMITIES (WRIST/ARM/LEG),NOT SUSTAINED,NaN,NaN
1161884,32132.0,BLACK,MALE,1961.0,70656.0,1997-08-04,Non-Violent,POLICE OFFICER,NaN,NaN,NaN,NaN
501888,14592.0,WHITE,MALE,1977.0,49548.0,2000-06-19,Non-Violent,POLICE OFFICER,THEFT,SUSTAINED,SEPARATION,NaN
1180880,32259.0,WHITE,MALE,1981.0,68262.0,2004-10-25,Non-Violent,POLICE OFFICER,CIVIL SUITS - THIRD PARTY,NOT SUSTAINED,NaN,NaN


### Backdoor

In [11]:
selected_columns[selected_columns['final_finding'] == 'SUSTAINED'].penalty_code.value_counts()

penalty_code
SUSPENSION         182798
SEPARATION          24894
REPRIMAND           17043
VIOLATION NOTED     10766
Name: count, dtype: int64

In [12]:
sustained_rows = selected_columns[selected_columns['final_finding'] == 'SUSTAINED']
sustained_rows

,link_UID,race,gender,birth_year,salary,appointed_date,violence_classification,cleaned_rank_x,allegation_category,final_finding,penalty_code,number_of_days
42,24345.0,HISPANIC,MALE,1967.0,90540.0,1992-12-07,Non-Violent,DETECTIVE,USE OF PROFANITY,SUSTAINED,REPRIMAND,NaN
43,24345.0,HISPANIC,MALE,1967.0,90540.0,1992-12-07,Non-Violent,DETECTIVE,USE OF PROFANITY,SUSTAINED,REPRIMAND,NaN
44,24345.0,HISPANIC,MALE,1967.0,90540.0,1992-12-07,Non-Violent,DETECTIVE,CONDUCT UNBECOMING,SUSTAINED,REPRIMAND,NaN
45,24345.0,HISPANIC,MALE,1967.0,90540.0,1992-12-07,Non-Violent,DETECTIVE,USE OF PROFANITY,SUSTAINED,REPRIMAND,NaN
46,24345.0,HISPANIC,MALE,1967.0,90540.0,1992-12-07,Non-Violent,DETECTIVE,NEGLECT OF DUTY,SUSTAINED,REPRIMAND,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1208698,32465.0,BLACK,FEMALE,1968.0,75372.0,1999-11-29,Violent,POLICE OFFICER,CLOSED HAND STRIKE (PUNCH),SUSTAINED,SEPARATION,NaN
1208699,32465.0,BLACK,FEMALE,1968.0,75372.0,1999-11-29,Non-Violent,POLICE OFFICER,"ASSAULT / BATTERY, ETC.",SUSTAINED,SEPARATION,NaN
1208700,32465.0,BLACK,FEMALE,1968.0,75372.0,1999-11-29,Non-Violent,POLICE OFFICER,INSUBORDINATION,SUSTAINED,SEPARATION,NaN
1208701,32465.0,BLACK,FEMALE,1968.0,75372.0,1999-11-29,Non-Violent,POLICE OFFICER,USE OF PROFANITY,SUSTAINED,SEPARATION,NaN


In [13]:
model_df = sustained_rows[['penalty_code','gender','race','allegation_category', 'cleaned_rank_x', 'violence_classification']]

In [16]:
model_df.head(10)

,penalty_code,gender,race,allegation_category,cleaned_rank_x,violence_classification
42,REPRIMAND,MALE,HISPANIC,USE OF PROFANITY,DETECTIVE,Non-Violent
43,REPRIMAND,MALE,HISPANIC,USE OF PROFANITY,DETECTIVE,Non-Violent
44,REPRIMAND,MALE,HISPANIC,CONDUCT UNBECOMING,DETECTIVE,Non-Violent
45,REPRIMAND,MALE,HISPANIC,USE OF PROFANITY,DETECTIVE,Non-Violent
46,REPRIMAND,MALE,HISPANIC,NEGLECT OF DUTY,DETECTIVE,Non-Violent
47,REPRIMAND,MALE,HISPANIC,PUSH/PULL/GRAB,DETECTIVE,Violent
48,REPRIMAND,MALE,HISPANIC,USE OF PROFANITY,DETECTIVE,Non-Violent
49,REPRIMAND,MALE,HISPANIC,USE OF PROFANITY,DETECTIVE,Non-Violent
50,REPRIMAND,MALE,HISPANIC,CONDUCT UNBECOMING,DETECTIVE,Non-Violent
51,REPRIMAND,MALE,HISPANIC,USE OF PROFANITY,DETECTIVE,Non-Violent


In [17]:
model_df.penalty_code.value_counts()

penalty_code
SUSPENSION         182798
SEPARATION          24894
REPRIMAND           17043
VIOLATION NOTED     10766
Name: count, dtype: int64

In [18]:
model_df['penalized'] = model_df['penalty_code'].apply(lambda x: 1 if x in ['SUSPENSION', 'SEPARATION'] else 0)

<ipython-input-18-4d1e7d261f8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['penalized'] = model_df['penalty_code'].apply(lambda x: 1 if x in ['SUSPENSION', 'SEPARATION'] else 0)


In [19]:
model_df['white'] = model_df['race'].apply(lambda x: 1 if x == 'WHITE' else 0)

<ipython-input-19-0afcff4228ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['white'] = model_df['race'].apply(lambda x: 1 if x == 'WHITE' else 0)


In [20]:
model_df['male'] = model_df['gender'].apply(lambda x: 1 if x == 'MALE' else 0)

<ipython-input-20-307361f5afdf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['male'] = model_df['gender'].apply(lambda x: 1 if x == 'MALE' else 0)


In [21]:
model_df['Violent'] = model_df['violence_classification'].apply(lambda x: 1 if x == 'Violent' else 0)

<ipython-input-21-5ec61e2009e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['Violent'] = model_df['violence_classification'].apply(lambda x: 1 if x == 'Violent' else 0)


In [36]:
model_df['NonViolent'] = model_df['violence_classification'].apply(lambda x: 1 if x == 'Non-Violent' else 0)

In [37]:
rank_order = {
    'CIVILIAN': 1,
    'POLICE OFFICER': 2,
    'FIELD TRAINING OFFICER': 3,
    'INVESTIGATOR': 4,
    'DETECTIVE': 5,
    'SERGEANT': 6,
    'LIEUTENANT': 7,
    'CAPTAIN': 8,
    'COMMANDER': 9,
    'DEPUTY CHIEF': 10
}

In [38]:
model_df['rank'] = model_df['cleaned_rank_x'].map(rank_order)

In [39]:
model_df['penalty_code'].value_counts(dropna = False)

penalty_code
SUSPENSION         182798
SEPARATION          24894
REPRIMAND           17043
NaN                 13085
VIOLATION NOTED     10766
Name: count, dtype: int64

In [40]:
def bootstrap(df, function, n=1000, ci=95, **kwargs):

    np.random.seed(42)  # Keep this random seed call for reproducibility

    estimates = []

    for _ in range(n):
        sample_df = df.sample(n=len(df), replace=True)
        estimates.append(function(sample_df, **kwargs))

    lower_bound = (100 - ci) / 2
    upper_bound = 100 - lower_bound

    ci_low = np.percentile(estimates, lower_bound, axis=0)
    ci_high = np.percentile(estimates, upper_bound, axis=0)

    return np.array([ci_low, ci_high])

In [41]:
model_df['rank'].value_counts()

rank
2     206837
6      22464
5      13559
3       3071
7       1701
9        526
4        311
10       100
8         13
1          4
Name: count, dtype: int64

In [54]:
import statsmodels.formula.api as smf

formula = 'penalized ~  white + male + Violent'

models = []

results = []

for rank in range(1,11):
    if rank in [1, 8, 10]:
        continue

    print(rank)

    curr_slice = model_df[model_df['rank'] == rank]
    curr_model = smf.logit(formula, curr_slice).fit()

    pred = curr_model.predict(model_df)

    results.append(pred.mean())

2
Optimization terminated successfully.
         Current function value: 0.420441
         Iterations 6
3
Optimization terminated successfully.
         Current function value: 0.304639
         Iterations 8
4
         Current function value: 0.320983
         Iterations: 35


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


5
Optimization terminated successfully.
         Current function value: 0.442290
         Iterations 7
6
Optimization terminated successfully.
         Current function value: 0.504638
         Iterations 6
7
Optimization terminated successfully.
         Current function value: 0.630028
         Iterations 5
9
         Current function value: 0.626775
         Iterations: 35


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [55]:
rank_order = {
    'CIVILIAN': 1,
    'POLICE OFFICER': 2,
    'FIELD TRAINING OFFICER': 3,
    'INVESTIGATOR': 4,
    'DETECTIVE': 5,
    'SERGEANT': 6,
    'LIEUTENANT': 7,
    'CAPTAIN': 8,
    'COMMANDER': 9,
    'DEPUTY CHIEF': 10
}

In [57]:
results #estimates for 2 3 4 5 6 7 9

[0.8455581255907577,
 0.739484579090697,
 0.7978488543527796,
 0.7920371459345236,
 0.7762155573760582,
 0.6688325525652726,
 0.442389981643277]